In [1]:
# An implementation of logistics regression for binary class labels

In [2]:
# imports
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np 
from io import BytesIO

import torch
import torch.nn.functional as F

In [5]:
# preparing a toy dataset
#DATASET
ds = np.lib.DataSource()
fp = ds.open('https://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data')

x = np.genfromtxt(ByteIO(fp.read().encode()), delimeter=',', usecols=range(2), max_rows=100)
y = np.zeros(100)
y[50:] = 1

np.random.seed(1)
idx = np.arange(y.shape[0])
np.random.shuffle(idx)
X_test, y_test = x[idx[:25]], y[idx[:25]]
X_train , y_train = x[idx[25:]], y[idx[25:]]
mu, std = np.mean(X_train, axis=0), np.std(X_train, axis=0)
X_train, X_test = (X_train - mu)/ std, (X_test - mu)/ std

# make some plots
fig, ax = plt.subplots(1,2, figsize=(7, 2.5))
ax[0].scatter(X_train[y_train == 1, 0], X_train[y_train == 1, 1])
ax[0].scatter(X_train[y_train == 0,0], X_train[y_train == 0,1])
ax[1].scatter(X_test[y_test == 1,0], X_test[y_test == 1,1])
ax[1].scatter(X_test[y_test == 0,0], X_test[y_test == 0,1])
plt.show()

OSError: https://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data not found.